In [23]:
from model.rnn import GRUDecoder
from model.autoencoder import AutoEncoder
import torch        

In [24]:
def get_device():
    # Check if CUDA is available
    if torch.cuda.is_available():
        # If CUDA is available, select the first CUDA device
        device = torch.device("cuda:0")
        print("Using CUDA device:", torch.cuda.get_device_name(0))
    # Check for MPS availability on supported macOS devices (requires PyTorch 1.12 or newer)
    elif torch.backends.mps.is_available():
        # If MPS is available, use MPS device
        device = torch.device("mps")
        print("Using MPS (Metal Performance Shaders) device")
    else:
        # Fallback to CPU if neither CUDA nor MPS is available
        device = torch.device("cpu")
        print("Using CPU")
    return device


## Autoencoder

In [25]:
### Initialization of the Autoencoder 
SEQ_LEN = 3000
HIDDEN_DIM = 512
ENCODING_SIZE = 64
model = AutoEncoder(vocab_size=100, embedding_size=HIDDEN_DIM, encoding_size=ENCODING_SIZE, sequence_len=SEQ_LEN)


/Users/carlomarx/Library/Python/3.9/lib/python/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [26]:
#let's assume we have a batch of 2 people
x = torch.randint(1,99, size=(2,SEQ_LEN))
y = model(x) 
## returns the original shape


In [27]:
### only to use the encoder part 
y = model.encode(x) # here y contains embedding of a survey per row

## Squeeze - Excite Networks

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SqueezeExcitation(nn.Module):
    def __init__(self, num_columns, hidden_size, reduction_ratio=16):
        super(SqueezeExcitation, self).__init__()
        # Assuming the reduction happens across the hidden_size
        self.num_columns = num_columns
        self.reduced_size = max(1, hidden_size // reduction_ratio)
        
        self.squeeze = nn.AdaptiveAvgPool1d(1)  # Squeezes hidden_size to 1
        self.excitation = nn.Sequential(
            nn.Linear(num_columns, self.reduced_size),
            nn.ReLU(inplace=True),
            nn.Linear(self.reduced_size, num_columns),
            nn.Sigmoid()
        )

    def forward(self, x):
        # x shape: [batch_size, num_columns, hidden_size]
        # Squeeze operation
        z = self.squeeze(x.permute(0, 1,2))  # Change to [batch_size, hidden_size, num_columns] for pooling
        # z shape: [batch_size, hidden_size, 1]
        print(z.shape)
        # Excitation operation
        z = z.view(z.size(0), -1)  # Flatten [batch_size, hidden_size]
        print(z.shape)
        s = self.excitation(z)  # [batch_size, num_columns]
        print(s.shape)
        s = s.view(s.size(0), s.size(1), 1)  # Reshape to [batch_size, num_columns, 1] to match original dimensions
        
        return x * s  # Apply recalibration weights to the original input

# Example usage
se_block = SqueezeExcitation(SEQ_LEN, HIDDEN_DIM)

# Example input
output_tensor = se_block(model.embedding(x))
print(output_tensor.shape)  # Should be [5, 10, 512]


torch.Size([2, 3000, 1])
torch.Size([2, 3000])
torch.Size([2, 3000])
torch.Size([2, 3000, 512])


In [16]:
y

tensor([[1.0531, 1.0730, 1.2044, 1.0634, 1.0802, 1.0725, 1.0801, 1.0112, 1.0577,
         1.1093, 1.1426, 0.8842, 1.0742, 1.2163, 1.2270, 1.1978, 1.1072, 0.9584,
         1.1679, 1.1188, 1.1647, 1.1729, 1.2542, 1.1120, 0.9967, 1.0776, 1.0980,
         1.1687, 1.1266, 0.9934, 0.9679, 1.1616, 1.0662, 0.9413, 1.0701, 1.0496,
         1.0380, 1.1257, 0.9062, 1.0719, 1.1047, 0.9403, 1.2135, 1.0926, 1.1796,
         1.0396, 0.9750, 1.1150, 0.8920, 1.0043, 1.0607, 1.0112, 0.9216, 1.0439,
         1.0199, 1.0194, 1.0808, 1.2047, 1.1377, 1.0877, 1.1994, 1.2468, 1.1343,
         1.1176],
        [1.3957, 1.2013, 1.2500, 1.0567, 1.0101, 1.1571, 1.0866, 1.0422, 1.1510,
         1.0925, 1.1267, 1.1172, 1.0557, 1.0159, 1.1122, 1.1162, 1.1513, 1.0397,
         0.8504, 0.9791, 1.0505, 1.0290, 0.9558, 1.1525, 1.0346, 1.0656, 1.0625,
         1.1594, 1.1062, 1.0022, 1.0685, 1.1871, 1.0504, 1.0977, 0.9951, 1.0144,
         0.9501, 1.1131, 1.0390, 1.1674, 1.1038, 1.0137, 1.1603, 1.0050, 1.1031,
         1

## RNN

In [28]:
## 
# input_size -> the size of the embedding of the autoencoder model
# hidden_size -> the size of the RNN to use in the decoder (the input_size and hidden_size can be different)
model = GRUDecoder(input_size=6, hidden_size=10, max_seq_len=4).to(get_device())

Using MPS (Metal Performance Shaders) device


In [8]:
# This is just an example

MAX_SEQ_LEN = 4 # max number of surveyas a person (in our dataset can have)
INPUT_SIZE = 6 # hidden dimmensions of autoencodder.

# let's say we have a person who only have 2 surveys
x0 = torch.rand(INPUT_SIZE) # embedding for the 1st survey 
x1 = torch.rand(INPUT_SIZE) # embedding for the 2nd survey

# the tensor for the person should be on the shape [MAX_SEQ_LEN, INPUT_SIZE]

e = torch.zeros(MAX_SEQ_LEN, INPUT_SIZE)
e[0] = x0
e[1] = x1
e = e.to(get_device()) # so this is a tensor for the person
#we also need to specify that the sequence has 'empty' embeddings
mask = torch.BoolTensor([True, True, False, False]).to(get_device()) # the last two dimensions are empty
## it is important that you append existing survey embeddings right next to each other (even if the year is missign between them, they should be still appended one after another)

## let assume we have a batch of people, I am reusing the same person, but in the pipeline is should be different people
# the batch size is 3 here 

x = torch.stack([e,e,e])
mask = torch.stack([mask, mask, mask])

Using MPS (Metal Performance Shaders) device
Using MPS (Metal Performance Shaders) device


In [9]:
xx = model(x, mask)

In [10]:
torch.nn.functional.sigmoid(xx)

tensor([[0.6218],
        [0.4513],
        [0.4098]], device='mps:0', grad_fn=<SigmoidBackward0>)